In [1]:
from resnet_dataset import ResnetImageDataset, get_class_str
from resnet_shuffled import ShuffledResnetVariable

from torchvision.models import resnet18, resnet50, ResNet50_Weights, ResNet18_Weights
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import os
import csv

from tqdm import tqdm


c:\Users\Leonard\anaconda3\envs\pytorch_nn_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CONFIG = {
    "dataset":{
        "img_dir": "C:/Users/Leonard/Desktop/DP/early_MLP_implementations/data/resnet18_set/images_unpacked/",
        "img_num": 1000,
    },
    "dataloader":{
        "batch_size": 16,
        "shuffle": False,
    },
}

In [3]:
def test_accuracy(model, dataloader):
    model.eval()
    model = model.to("cuda" if torch.cuda.is_available() else "cpu")

    correct_total = 0
    sample_count = 0

    dataloader = tqdm(dataloader, total=len(dataloader))
    
    for images, labels in dataloader:
        images, labels = images.to("cuda" if torch.cuda.is_available() else "cpu"), labels.to("cuda" if torch.cuda.is_available() else "cpu")
        pred = model.forward(images)
        
        probabilities = torch.nn.functional.softmax(pred, dim=1)
        topk_prob, topk_catid = torch.topk(probabilities, 1)

        topk_catid = torch.squeeze(topk_catid, dim=1).type(torch.float64)
        correct = (labels == topk_catid).sum().item()
        correct_total += correct

        sample_count += len(images)
    return correct_total/sample_count


In [4]:
transform_pipe = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
test_dataset = ResnetImageDataset(img_dir=CONFIG['dataset']['img_dir'], img_num_cap=CONFIG['dataset']['img_num'], transform=transform_pipe)
test_dataloader = DataLoader(test_dataset, batch_size=CONFIG['dataloader']['batch_size'], shuffle=CONFIG['dataloader']['shuffle'])

print(f"The directory contains {len(os.listdir(CONFIG['dataset']['img_dir']))} image files")
print(f"We are using {CONFIG['dataset']['img_num']}/{len(os.listdir(CONFIG['dataset']['img_dir']))} images")

The dataset contains 643 unique classes
The directory contains 50000 image files
We are using 1000/50000 images


In [6]:
f = open('../data/accuracy_scores.csv', 'w')
writer = csv.writer(f)

header = ['model', 'layer', 'num_samples', 'num_shuffled', 'accuracy']
writer.writerow(header)



47

In [7]:
res50 = resnet50(weights=ResNet50_Weights.DEFAULT)

shuffled_res = ShuffledResnetVariable(model=res50)

In [8]:
for i in range(0, 2049, 32):
    shuffled_res.change_n_shuffled(i)
    accuracy = test_accuracy(model=shuffled_res, dataloader=test_dataloader)

    row = ["resnet50", "fc", CONFIG["dataset"]["img_num"], i, accuracy]
    writer.writerow(row)
    print(row)


  3%|▎         | 1/32 [00:08<04:32,  8.80s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 98.00 MiB (GPU 0; 4.00 GiB total capacity; 3.26 GiB already allocated; 0 bytes free; 3.43 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
f.close()